In [5]:
# change two place
def get_balance_sheet(stock_id, start_year, end_year):
    import pandas as pd
    import sqlite3
    import time
    import requests
    
    balance_sheet_url = 'https://mops.twse.com.tw/mops/web/ajax_t164sb03'
    income_statement_url = 'https://mops.twse.com.tw/mops/web/ajax_t164sb04'
    cash_flows_url = 'https://mops.twse.com.tw/mops/web/ajax_t164sb05'
    
    url = balance_sheet_url # change 1
    
    year = range(start_year, end_year+1)
    season = range(1, 5)
    dfs = []
    for y in year:
        for s in season:
            form_data = {
                'encodeURIComponent': '',
                'step': '1',
                'firstin': '1',
                'off': '1',
                'queryName': 'co_id',
                'inpuType': 'co_id',
                'TYPEK': 'all',
                'isnew': 'false',
                'co_id': str(stock_id),
                'year': str(y),
                'season': str(s),
            }

            res = requests.post(url, data=form_data)
            table = pd.read_html(res.text)[1]
            table = table.iloc[:, 0:2]
            table.columns = table.columns.get_level_values(2)
            table = table.set_index('會計項目')
            table = table[table.duplicated() == False]
            dfs.append(table)
            time.sleep(3)
            
    df_final = pd.DataFrame(columns=['會計項目'])
    for df in dfs:
        df_final = df_final.merge(df, how='outer', left_index=True, right_index=True)

    df_final = df_final[df_final.index.duplicated() == False]
    df_final = df_final.T.reset_index()
    df_final = df_final.rename(columns={'index': 'date', '會計項目': 'index'})
    df_final = df_final.shift(-1)
    last_index = df_final.index.stop - 1
    df_final = df_final.drop(axis=0, index=last_index)
    conn = sqlite3.connect('balance_sheet.db') # change 2
    df_final.to_sql('balance_' + str(stock_id), conn, if_exists='replace') # change 3
    return df_final

In [6]:
download_table = [1101, 2317, 2330, 2498]
for d in download_table:
    table = get_balance_sheet(d, 108, 109)
# table
# table_2317 = get_balance_sheet(2317, 102, 104)
# table_2317